In [73]:
# df_x will have:
# Cutoff:
# genetic	 exercise smoking alcohol lifespan BMI

# not cutoff:
# mass

# Not included:
# length sugar

# Length will be required in order to calculate BMI.

In [74]:
# Imports
import logging
import pandas as pd
# import requests
import sqlite3
import sklearn
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
from scipy.stats import pearsonr




#logging info
logging.basicConfig(level = logging.DEBUG)



dbName = "../rest_server/medisch_centrum_randstad/db.sqlite3"
tableName = "rest_api_netlify"
url = "http://localhost:8080/medish_centrum_randstad/api/netlify?page=1"
csvFile = "rest_server_new/medisch_centrum_randstad/data/data.csv"

##########################
### Read from REST API ###
##########################
# logging.info("Extract from REST API")
# response = requests.get(url)
# file_contents= response.json()  #dictionary
# logging.info("Load API Response into Pandas dataframe")
# df = pd.DataFrame.from_dict(file_contents['data']) #all the needed info was condensed into one data column called 'data'
# df3 = df.copy() #keep original for df3 
# logging.debug(df.head())
# logging.info("Load dataset in Database")

######################
### Read from .csv ###
######################
#df pd.read_csv('csvFile',skipinitialspace=True)
#df3 = df.copy() #keep original for df3 
#logging.info("Load dataset in Database")

#


In [75]:
###########################
### Read from SQlite3 db created in our pipeline ###
############################
dbConnection = sqlite3.connect(dbName)

#query db and write to pd:
dfFromDB = pd.read_sql_query(f"SELECT * FROM {'df2'}", dbConnection)
#sql adds index, remove:
df2 = dfFromDB.drop('index', axis=1)
pd.set_option('display.max_columns', 10)
display(df2.head(2))
# display(df2.info)

df=df2.copy()
y_axis_test = 'lifespan'

df_x=df[y_axis_test].copy()

dbConnection.close()

,genetic,length,mass,exercise,smoking,alcohol,lifespan,sugar,BMI
0,73.9,185.0,99.7,0.9,0.0,2.4,73.1,6.9,29.1
1,86.0,172.0,105.4,1.8,8.1,0.4,85.0,4.2,35.6


In [90]:
def P_value(df_1, X, y, df_2):
    # Calculate the Pearson correlation coefficient and p-value
    corr, p_value = pearsonr(df_1[X], df_1[y])
    
    # Check if the p-value is less than 0.05
    if p_value < 0.05:
        df_2[X] = df_1[X].copy()  # Add X from data frame d to data frame df_x
        print(f'The P_value is {p_value}. The mean distribution of {X} compared to {y} is suitable for linear regression')
        return df_2[X]
    else:
        print(f'The P_value is {p_value}. The mean distribution of {X} compared to {y} is NOT suitable for linear regression')
        return df_1[X]

In [77]:
# loop door de df om te kijken welke colomen in aanmerkig komen voor lineare regressie met de geschreven functie P_value. en om deze in een niewe df_x te zetten



y_axis_test = 'lifespan'

for col in df.columns:
    if col != y_axis_test:
        df = P_value(df, col, y_axis_test,df_x)
        



The P_value is 0.0. The mean distribution of genetic compared to lifespan is suitable for linear regression
The P_value is 0.40064820230526876. The mean distribution of length compared to lifespan is NOT suitable for linear regression
The P_value is 2.4035955448719514e-05. The mean distribution of mass compared to lifespan is suitable for linear regression
The P_value is 1.0912436283980197e-07. The mean distribution of exercise compared to lifespan is suitable for linear regression
The P_value is 2.0223231220460298e-57. The mean distribution of smoking compared to lifespan is suitable for linear regression
The P_value is 4.835951231696475e-05. The mean distribution of alcohol compared to lifespan is suitable for linear regression
The P_value is 0.4867028898760703. The mean distribution of sugar compared to lifespan is NOT suitable for linear regression
The P_value is 2.0719534641846765e-06. The mean distribution of BMI compared to lifespan is suitable for linear regression


In [78]:
display (df_x.head())

0    73.1
1    85.0
2    81.6
3    81.0
4    75.0
Name: lifespan, dtype: object

In [81]:
df_cut=df2.copy()

# Computing IQR
Q1 = df_cut.quantile(0.25)
Q3 = df_cut.quantile(0.75)
IQR = Q3 - Q1

# Clipping the IQR*|15.*IQD|
df_cut = df_cut[~((df_cut < (Q1 - 1.5 * IQR)) |(df_cut > (Q3 + 1.5 * IQR))).any(axis=1)]

In [88]:
df5 = df_x ['lifespan','mass']
df5 = df5.join(df_cut['genetic','exercise','smoking','alcohol','BMI'])

display(df5.head())

KeyError: 'key of type tuple not found and not a MultiIndex'